In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import sys
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import re

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix,accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('./new.csv')
df = df.dropna(subset = ['comment_text']).reset_index(drop = True)
df = df.dropna(subset = ['toxic']).reset_index(drop = True)

In [3]:
X = df['comment_text'].values
y = df['toxic'].values

In [4]:
trainX,testX,trainY,testY = train_test_split(X,y,test_size = 0.2,random_state = 0)

In [5]:
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
# print(len(all_stopwords))
cv = CountVectorizer(stop_words=all_stopwords)
Xtrain = cv.fit_transform(trainX)
# print(len(Xtrain[0]))

Xtest = cv.transform(testX)

In [6]:
print(df['toxic'].isnull().sum())

0


In [7]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(Xtrain,trainY)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [8]:
y_pred = classifier.predict(Xtest)
print(confusion_matrix(testY,y_pred))
print(accuracy_score(testY,y_pred))

[[28505   176]
 [ 1343  1866]]
0.9523675133270618


In [9]:
new_review = 'Go to the hell and die'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()

new_X_test = cv.transform([new_review])
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)

[1]


In [10]:
from sklearn.model_selection import cross_val_score
# ten test folds will be done out of which we get 10 accuracies ,we have to take avg
accuracies = cross_val_score(estimator = classifier,X = Xtrain,y = trainY,cv = 10) # cv - number of folds
print("Accuracy : {:.2f} %".format(accuracies.mean()*100)) # mean of 10 accuracies
print("Standard deviation : {:.2f} %".format(accuracies.std()*100))

Accuracy : 95.14 %
Standard deviation : 0.14 %
